In [1]:
#!python -m spacy download pt_core_news_lg
#!python -m spacy download en_core_web_lg
#%pip install -U scikit-learn

In [1]:
import os
import io
import stardog
import pandas as pd
import spacy
from spacy import displacy
from sklearn.cluster import DBSCAN
import numpy as np
import requests
from unidecode import unidecode
import re
import json
from langdetect import detect

### Specify Stardog connection details

In [36]:
# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

In [37]:
database_name = 'IndigenousSlavery'
conn = stardog.Connection(database_name, **connection_details)

In [46]:
# Query que busca as teses e os seus abstracts. Foi incluído um filtro para buscar apenas 
# os abstract em português, inglês ou em outra língua
query = """
SELECT ?thesis ?abstract (lang(?abstract) AS ?lang) WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <http://purl.org/ontology/bibo/abstract> ?abstract.
  FILTER (lang(?abstract) IN("pt", "en"))
  
}

"""

query2 = """
SELECT ?thesis ?ent  WHERE {
  ?thesis a <http://purl.org/ontology/bibo/Thesis>.
  ?thesis <https://schema.org/mentions> ?ent.  
}

"""

query3 = """
SELECT ?place   WHERE {
  ?place a <https://schema.org/Place>. 
}

"""
# FILTER (langMatches(lang(?abstract),"pt"))
# FILTER (langMatches(lang(?abstract),"en"))
# FILTER (lang(?abstract) NOT IN("pt", "en"))
# LIMIT 20

csv_results = conn.select(query3, content_type='text/csv')
thesis_abstract = pd.read_csv(io.BytesIO(csv_results))
thesis_abstract


,place


In [5]:
# Extraíndo as entidades dos abstracts

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

#Dicionário que receberá as instâncias de cada tese
instances_dic = {}

for n in range(len(thesis_abstract)):

    # Processando os abstracts em português
    if thesis_abstract['lang'][n] == 'pt':

        doc_pt = nlp_pt(str(thesis_abstract['abstract'][n]))
        persons = []
        gpes = []
        #orgs = []
        for ent in doc_pt.ents:
            if ent.label_ == "PER": #"PERSON":
                persons.append(ent)
            if ent.label_ == "LOC": #"GPE":
                gpes.append(ent)
            #if ent.label_ == "ORG":
            #    orgs.append(ent)

        #instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'ORG':orgs, 'lang': 'pt'}
        instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'lang': 'pt'}

    # Processando os abstracts em inglês
    if thesis_abstract['lang'][n] == 'en':

        doc_en = nlp_en(str(thesis_abstract['abstract'][n]))
        persons = []
        gpes = []
        #orgs = []
        for ent in doc_en.ents:
            if ent.label_ == "PERSON":
                persons.append(ent)
            if ent.label_ == "GPE":
                gpes.append(ent)
            #if ent.label_ == "ORG":
            #    orgs.append(ent)

        #instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'ORG':orgs, 'lang': 'en'}
        instances_dic[thesis_abstract['thesis'][n].replace('tag:stardog:api:','')] = {'PER':persons, 'LOC':gpes, 'lang': 'en'}

    if n%500 == 499:
        print(n, " teses processadas.")
    #    break

499  teses processadas.
999  teses processadas.
1499  teses processadas.
1999  teses processadas.
2499  teses processadas.
2999  teses processadas.
3499  teses processadas.
3999  teses processadas.
4499  teses processadas.
4999  teses processadas.
5499  teses processadas.
5999  teses processadas.
6499  teses processadas.
6999  teses processadas.
7499  teses processadas.
7999  teses processadas.
8499  teses processadas.
8999  teses processadas.
9499  teses processadas.
9999  teses processadas.
10499  teses processadas.
10999  teses processadas.
11499  teses processadas.
11999  teses processadas.
12499  teses processadas.
12999  teses processadas.
13499  teses processadas.


### Rascunho

In [6]:
# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))

In [ ]:
# Criando URIs para as entidades
for thes in instances_dic:
    for type_ent in ['LOC', 'PER']:
        list_ent = []
        for span in instances_dic[thes][type_ent]:
            list_ent.append(process_uri(span.text))
        instances_dic[thes][type_ent + '_uri'] = list(set(list_ent))

Preparando o texto em turtle

In [38]:
# Prefixos
prefixos = """ @prefix ns: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix event: <http://purl.org/NET/c4dm/event.owl#> .
@prefix prism: <http://prismstandard.org/namespaces/1.2/basic/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schemas.talis.com/2005/address/schema#> .
@prefix status: <http://purl.org/ontology/bibo/status/> .
@prefix degrees: <http://purl.org/ontology/bibo/degrees/> .
@prefix stardog: <tag:stardog:api:> .
@base <http://www.w3.org/2002/07/owl#> .

"""

Conectando com o banco e adicionando as triplas

In [42]:
# Funcão que recebe os prefixos e triplas e as carrega à base de dados
def add_triplas_to_stardog(prefixos, triplas):

    # Incluindo prefixos às triplas
    triplas = prefixos + " " + triplas

    ### Connect to the Stardog database
    database_name = 'IndigenousSlavery'
    conn = stardog.Connection(database_name, **connection_details)

    conn.begin()
    conn.add(stardog.content.Raw(triplas, 'text/turtle'))
    conn.commit() # commit the transaction

Preparando as triplas para conectar entidades às teses e carregando à base.

In [48]:
loc_uri = []
for thes in instances_dic:
    for loc in instances_dic[thes]['LOC_uri']:
        loc_uri.append(loc)

triplas = """ """
for loc in (set(loc_uri)):
    triplas = triplas + " " + """
    stardog:""" + loc + """ rdf:type <https://schema.org/Place>."""

add_triplas_to_stardog(prefixos, triplas)

In [28]:
per_uri = []
for thes in instances_dic:
    for per in instances_dic[thes]['PER_uri']:
        per_uri.append(per)

triplas = """ """
for per in (set(per_uri)):
    triplas = triplas + " " + """
    stardog:""" + per + """ rdf:type foaf:Person."""

add_triplas_to_stardog(prefixos, triplas)

In [49]:
triplas = """ """
n = 0
for thes in instances_dic:
    n = n + 1
    for loc in instances_dic[thes]['LOC_uri']:
        tripla = """
        stardog:""" + thes + """ <https://schema.org/mentions> stardog:""" + loc + """.
        """
        triplas = triplas + " " + tripla

    for per in instances_dic[thes]['PER_uri']:
        tripla = """
        stardog:""" + thes + """ <https://schema.org/mentions> stardog:""" + per + """.
        """  
        triplas = triplas + " " + tripla
    
add_triplas_to_stardog(prefixos, triplas)

### Fim do rascunho

In [23]:
# Para uma lista de entidades verificar se são a mesma instância e qual o termo mais comum para representá-las

def entidades_consolidadas(list_ent, th):
    
    if list_ent == []:
        return None

    # lista com os vetores das entidades
  
    list_vectors = np.array([i.vector for i in list_ent])

    #clusterizando os vetores de acordo com threshold th
    clustering = DBSCAN(eps=1-th, min_samples=1, metric='cosine').fit(list_vectors)

    #processando os clusters 
    ents_dic ={}

    for i in set(clustering.labels_):
        clus_index = np.where(clustering.labels_ == i)[0]
        label = []
        vec = []
        
        for c in clus_index:
            vec.append(list_ent[c].vector)
            label.append(list_ent[c].text)

        ents_dic[max(set(label), key=label.count)] = {'labels': list(set(label))} #, 'vector': np.average(vec, axis=0).tolist()}

    return ents_dic



In [7]:
# Reprocessando as listas de entidades para eliminar as entidades duplicadas de cada tese e obtendo o vetor de cada entidade
th = 0.90

i = 0
for key in instances_dic:
    for type_ent in instances_dic[key]:
        if type_ent != 'lang':
            list_ent = instances_dic[key][type_ent]
            instances_dic[key][type_ent] = entidades_consolidadas(list_ent, th)


In [2]:
# Salvando o dicionário com a lista de entidades mencionadas no abstract
#with open('data/abstract_NER.json', 'w') as fp:
#    json.dump(instances_dic, fp)

# Abrindo os dicionários salvos anteriormente
with open('data/abstract_NER.json') as fp:
    instances_dic = json.load(fp)

In [9]:
"""
# Clusterizando as entidades em Português

per_list = []
per_label_list = []
per_vector_list = []
per_key_list = []

for key in instances_dic:
    if instances_dic[key]['lang'] == 'pt':
        if instances_dic[key]['PER'] != None:
            for p in instances_dic[key]['PER']:
                per_list.append(p)
                per_label_list.append(instances_dic[key]['PER'][p]['labels'])
                per_vector_list.append(instances_dic[key]['PER'][p]['vector'])
                per_key_list.append(key)
"""

In [10]:
"""th =0.98
clustering_per = DBSCAN(eps=1-th, min_samples=1, metric='cosine').fit(per_vector_list)"""

In [53]:
"""per_citadas = pd.DataFrame({'teses': per_key_list, 'person': per_list, 'labels' :per_label_list, 'cluster': clustering_per.labels_})
per_citadas[per_citadas['cluster'] == 140]
#len(set(clustering_per.labels_))
#per_label_list[[clustering_per.labels_ == 0]]"""


"per_citadas = pd.DataFrame({'teses': per_key_list, 'person': per_list, 'labels' :per_label_list, 'cluster': clustering_per.labels_})\nper_citadas[per_citadas['cluster'] == 140]\n#len(set(clustering_per.labels_))\n#per_label_list[[clustering_per.labels_ == 0]]"

Coletando dados da DBPEDIA

In [3]:
# Coletando todas as labels das entidades extraídas dos abstracts

people_labels = []
local_labels = []
org_labels = []

for key in instances_dic:
    for type_ent in instances_dic[key]:

        list_ent = instances_dic[key][type_ent]
        if list_ent != None:
            for ent in list_ent:
                if type_ent == 'PER':
                    people_labels = people_labels + list_ent[ent]['labels']
                if type_ent == 'LOC':
                    local_labels = local_labels + list_ent[ent]['labels']
                if type_ent == 'ORG':
                    org_labels = org_labels + list_ent[ent]['labels']

people_labels = list(set(people_labels))
local_labels = list(set(local_labels))
org_labels = list(set(org_labels))

print('Total de labels de ORG: ', len(org_labels))
print('Total de labels de PER: ', len(people_labels))
print('Total de labels de LOC: ', len(local_labels))

Total de labels de ORG:  14472
Total de labels de PER:  12564
Total de labels de LOC:  14126


In [4]:
# Funcao para buscar as labels na DBPEDIA. A funcao retorna as URI a abstracts de entidades que estao registradas na DBPEDIA.
# sc_th é o threshold do score de busca para uma URI da DBPEDIA ser retornada.
def DBPEDIA_search(labels, classe, sc_th):

    dbpedia_ent = []

    for ent_bruto in labels:
        # processando as labels para ser buscada na DBPEDIA
        ent = re.sub('[^a-zA-Z0-9_ ]', '', unidecode(ent_bruto)).strip()
        ent = ent.replace(' and ', ' ').replace(' or ', ' ')
        ent = re.sub("\s\s+" , " ", ent)
        
        ent_vector = "'" + ent.replace(" ", "', '") + "'"
        ent_contains = ent.replace(" ", " AND ")

        # Sparql query para fazer busca em linguagem natural e retornar os resultados rankeados pelo score ?sc.
        query = """ 
            define input:ifp "IFP_OFF"  
            select ?s1 as ?c1, (bif:search_excerpt (bif:vector (""" + ent_vector + """), ?o1)) as ?c2, ?sc, ?rank, ?g, ?abstract 
            where 
            { 
            select ?s1, (?sc * 3e-1) as ?sc, ?o1, (sql:rnk_scale (<LONG::IRI_RANK> (?s1))) as ?rank, ?g, ?abstract 
            
            where  
            { 
                quad map virtrdf:DefaultQuadMap 
                { 
                graph ?g 
                { 
                    ?s1 ?s1textp ?o1 .
                    ?o1 bif:contains  '(""" + ent_contains + """)'  option (score ?sc)  .
                    ?s1 a """ + classe + """.
                    ?s1 dbo:abstract ?abstract.
                    FILTER (lang(?abstract) IN("pt", "en"))
                }
                } 
            }

            order by desc (?sc * 3e-1 + sql:rnk_scale (<LONG::IRI_RANK> (?s1)))  limit 5  offset 0 
            } 
            """
        # URL da DBPEDIA e request
        url = 'http://dbpedia.org/sparql'

        try:
            r = requests.get(url, params = {'format': 'json', 'query': query})
            data = r.json()

            # processando os resultados obtidos
            if data['results']['bindings'] != []:

                bindings = []
                for r in data['results']['bindings']:
                    if float(r['sc']['value']) > sc_th:
                        bindings.append((ent_bruto, r['sc']['value'], r['c1']['value'], r['abstract']['value']))
                
                dbpedia_ent = dbpedia_ent + list(set(bindings))
        except:
            print('Erro ao buscar a label: ', ent_bruto)
    return dbpedia_ent  



In [5]:
# Funcão que recebe uma lista de labels de uma determinada classe, 
# busca essas labels na DBPEDIA e salva as tuplas com as labels e abstracts no diretório desejado.

def coletando_dbpedia_tuplas(labels, classe, path):
    # Lista recebe as tuplas
    db_tupla = []
    #Bach em que as tuplas serão salvas
    step = 100
    for n in range(0, len(labels), step):
        # Busca as labels usando a funcão DBPEDIA_search
        ex_DB = DBPEDIA_search(labels[n:n+step], classe, 3.0)
        
        for ex in ex_DB:
            db_tupla.append((ex[2], ex[3]))
        # Elimina as tuplas repetidas
        db_tupla = list(set(db_tupla))

        # Salvando o dicionário com a lista de entidades mencionadas no abstract
        with open(path, 'w') as fp:
            json.dump(db_tupla, fp)
        print(n+step, '- Total de tuplas: ', len(db_tupla))

    return db_tupla

In [ ]:
coletando_dbpedia_tuplas(people_labels, 'dbo:Person', 'data/DBPEDIA_people.json')

In [ ]:
coletando_dbpedia_tuplas(local_labels, 'dbo:Place', 'data/DBPEDIA_locals.json')

In [ ]:
coletando_dbpedia_tuplas(org_labels, 'dbo:Organisation', 'data/DBPEDIA_orgs.json')

Criando as triplas e carregando no knowledge graph

In [4]:
# Prefixos
prefixos = """ @prefix ns: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix event: <http://purl.org/NET/c4dm/event.owl#> .
@prefix prism: <http://prismstandard.org/namespaces/1.2/basic/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schemas.talis.com/2005/address/schema#> .
@prefix status: <http://purl.org/ontology/bibo/status/> .
@prefix degrees: <http://purl.org/ontology/bibo/degrees/> .
@prefix stardog: <tag:stardog:api:> .
@base <http://www.w3.org/2002/07/owl#> .

"""

In [5]:
# Funcão que recebe os prefixos e triplas e as carrega à base de dados
def add_triplas_to_stardog(prefixos, triplas):

    # Incluindo prefixos às triplas
    triplas = prefixos + " " + triplas

    ### Connect to the Stardog database
    database_name = 'IndigenousSlavery'
    conn = stardog.Connection(database_name, **connection_details)

    conn.begin()
    conn.add(stardog.content.Raw(triplas, 'text/turtle'))
    conn.commit() # commit the transaction

In [6]:
# Abrindo os dicionários salvos anteriormente
with open('data/DBPEDIA_people.json') as fp:
    dbpedia_people = json.load(fp)

In [7]:
# Adicionando pessoas
triplas = """ """

for resource in dbpedia_people:
    tripla = """
    <""" + resource[0].replace('/resource/', '/page/') + """> rdf:type bibo:dbpedia.
    <""" + resource[0] + """> rdf:type foaf:Person.
    <""" + resource[0].replace('/resource/', '/page/') + """> terms:subject <""" + resource[0] + """>. 
    <""" + resource[0].replace('/resource/', '/page/') + """> bibo:abstract '""" + str(resource[1]).replace("'","").replace(u'\\', u' ') + """'@""" + detect(resource[1]) + """.
    """
    triplas = triplas + " " + tripla

In [8]:
add_triplas_to_stardog(prefixos, triplas)

In [9]:
# Abrindo os dicionários salvos anteriormente
with open('data/DBPEDIA_locals.json') as fp:
    dbpedia_locals = json.load(fp)

In [10]:
# Adicionando locais
triplas = """ """

for resource in dbpedia_locals:
    tripla = """
    <""" + resource[0].replace('/resource/', '/page/') + """> rdf:type bibo:dbpedia.
    <""" + resource[0] + """> rdf:type <https://schema.org/Place>.
    <""" + resource[0].replace('/resource/', '/page/') + """> terms:subject <""" + resource[0] + """>. 
    <""" + resource[0].replace('/resource/', '/page/') + """> bibo:abstract '""" + str(resource[1]).replace("'","").replace(u'\\', u' ') + """'@""" + detect(resource[1]) + """.
    """
    triplas = triplas + " " + tripla

In [11]:
add_triplas_to_stardog(prefixos, triplas)

In [12]:
# Abrindo os dicionários salvos anteriormente
with open('data/DBPEDIA_orgs.json') as fp:
    dbpedia_orgs = json.load(fp)

In [13]:
# Adicionando organizacões
triplas = """ """

for resource in dbpedia_orgs:
    tripla = """
    <""" + resource[0].replace('/resource/', '/page/') + """> rdf:type bibo:dbpedia.
    <""" + resource[0] + """> rdf:type foaf:Organization.
    <""" + resource[0].replace('/resource/', '/page/') + """> terms:subject <""" + resource[0] + """>. 
    <""" + resource[0].replace('/resource/', '/page/') + """> bibo:abstract '""" + str(resource[1]).replace("'","").replace(u'\\', u' ') + """'@""" + detect(resource[1]) + """.
    """
    triplas = triplas + " " + tripla

In [14]:
add_triplas_to_stardog(prefixos, triplas)